# TSA Chapter 9: Retail Sales

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch9_retail_sales/TSA_ch9_retail_sales.ipynb)

This notebook generates the simulated retail sales visualization with weekly and yearly seasonality plus holiday effects.

In [ ]:
!pip install numpy pandas matplotlib -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Color scheme
BLUE    = '#1A3A6E'
RED     = '#DC3545'
GREEN   = '#2E7D32'
ORANGE  = '#E67E22'
GRAY    = '#666666'
PURPLE  = '#8E44AD'

# Transparent backgrounds
plt.rcParams['figure.facecolor'] = 'none'
plt.rcParams['axes.facecolor'] = 'none'
plt.rcParams['savefig.facecolor'] = 'none'
plt.rcParams['savefig.transparent'] = True

# No top/right spines
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False

# General styling
plt.rcParams['axes.grid'] = False
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Helvetica', 'Arial', 'DejaVu Sans']
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['xtick.labelsize'] = 9
plt.rcParams['ytick.labelsize'] = 9
plt.rcParams['legend.fontsize'] = 9
plt.rcParams['legend.facecolor'] = 'none'
plt.rcParams['legend.framealpha'] = 0
plt.rcParams['axes.linewidth'] = 0.6
plt.rcParams['lines.linewidth'] = 1.0


def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    try:
        charts_path = os.path.join('..', '..', 'charts', name)
        fig.savefig(f'{charts_path}.pdf', bbox_inches='tight', transparent=True, dpi=150)
        fig.savefig(f'{charts_path}.png', bbox_inches='tight', transparent=True, dpi=150)
    except Exception:
        pass
    print(f'Saved: {name}.pdf + .png')


def legend_outside(ax, ncol=3):
    """Place legend outside bottom of plot."""
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.18), ncol=ncol, frameon=False)

In [ ]:
# Chart: ch9_retail_sales
# Simulated retail sales with weekly + yearly seasonality + holidays
np.random.seed(77)
n = 365 * 2
t = np.arange(n)
dates_retail = pd.date_range('2021-01-01', periods=n, freq='D')

trend = 500 + 0.5 * t
weekly_s = 50 * np.sin(2 * np.pi * t / 7)
yearly_s = 150 * np.sin(2 * np.pi * (t - 90) / 365)  # peak in spring

# Holiday spikes (Black Friday, Christmas)
holidays = np.zeros(n)
for yr_offset in [0, 365]:
    # Black Friday ~ day 328
    bf = 328 + yr_offset
    if bf < n:
        holidays[max(0, bf-2):min(n, bf+3)] = 300
    # Christmas ~ day 358
    xm = 358 + yr_offset
    if xm < n:
        holidays[max(0, xm-3):min(n, xm+1)] = 250

sales = trend + weekly_s + yearly_s + holidays + np.random.normal(0, 30, n)

fig, axes = plt.subplots(2, 1, figsize=(14, 7))

axes[0].plot(dates_retail, sales, color=BLUE, linewidth=0.5, label='Daily Sales')
axes[0].set_title('Retail Sales with Multiple Seasonality and Holidays', fontweight='bold')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Sales (units)')

# Mark holidays
for yr_offset in [0, 365]:
    bf = 328 + yr_offset
    xm = 358 + yr_offset
    if bf < n:
        axes[0].axvline(x=dates_retail[bf], color=RED, linestyle='--', alpha=0.5,
                        label='Black Friday' if yr_offset == 0 else '')
    if xm < n:
        axes[0].axvline(x=dates_retail[xm], color=GREEN, linestyle='--', alpha=0.5,
                        label='Christmas' if yr_offset == 0 else '')
legend_outside(axes[0], ncol=3)

# Decomposition-style view
axes[1].plot(dates_retail, weekly_s, color=ORANGE, linewidth=0.5, alpha=0.7, label='Weekly')
axes[1].plot(dates_retail, yearly_s, color=PURPLE, linewidth=1.5, label='Yearly')
axes[1].bar(dates_retail, holidays, color=RED, alpha=0.5, width=1, label='Holidays')
axes[1].set_title('Seasonal Components', fontweight='bold')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Effect')
legend_outside(axes[1], ncol=3)

plt.tight_layout(h_pad=3.0)
save_chart(fig, 'ch9_retail_sales')
plt.show()